In [ ]:
cycle = 1
seeding_dataset = 'data/ChemBL-35-cleaned.csv' # initialized with 50k-ChemBL.csv

trainingset_path  = 'data/'+str(cycle)+'-training_set_org.csv'  
charset_path= 'data/1-0.001-inp.h5'
latent_dataset_path = 'data/'+str(cycle)+'-training_set_latent_space.csv'
PATH = "checkpoint_239.pth"

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from group_selfies import (
    fragment_mols, 
    Group, 
    MolecularGraph, 
    GroupGrammar, 
    group_encoder
)

from rdkit.Chem import rdmolfiles
from rdkit.Chem.Draw import IPythonConsole

import IPython.display # from ... import display
from test_utils import *
from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*') 

import os
import sys
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
from rdkit.Chem import QED

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import gzip
import pandas as pd
import h5py
import numpy as np
from __future__ import print_function
import argparse
import os
import h5py
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn import model_selection


def one_hot_array(i, n):
    return map(int, [ix == i for ix in xrange(n)])

def one_hot_index(vec, charset):
    return map(charset.index, vec)

def from_one_hot_array(vec):
    oh = np.where(vec == 1)
    if oh[0].shape == (0, ):
        return None
    return int(oh[0][0])

def decode_smiles_from_indexes(vec, charset):
    # Ensure that each element in 'vec' is a string (not numpy.bytes_)
    return "".join(map(lambda x: str(charset[x], 'utf-8') if isinstance(charset[x], bytes) else charset[x], vec)).strip()

def load_dataset(filename, split = True):
    h5f = h5py.File(filename, 'r')
    if split:
        data_train = h5f['data_train'][:]
    else:
        data_train = None
    data_test = h5f['data_test'][:]
    charset =  h5f['charset'][:]
    h5f.close()
    if split:
        return (data_train, data_test, charset)
    else:
        return (data_test, charset)

class MolecularVAE(nn.Module):
    def __init__(self):
        super(MolecularVAE, self).__init__()

        self.conv_1 = nn.Conv1d(120, 9, kernel_size=9)
        self.conv_2 = nn.Conv1d(9, 9, kernel_size=9)
        self.conv_3 = nn.Conv1d(9, 10, kernel_size=11)
        self.linear_0 = nn.Linear(280, 435) # changed from 70 to 280 to reflect the change of charset size
        self.linear_1 = nn.Linear(435, 292)
        self.linear_2 = nn.Linear(435, 292)
        
        self.linear_3 = nn.Linear(292, 292)
        self.gru = nn.GRU(292, 501, 3, batch_first=True)
        self.linear_4 = nn.Linear(501, 54) # changed this output from 33 to 54 to reflect the larger charset size
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()

    def encode(self, x):
        x = self.relu(self.conv_1(x))
        x = self.relu(self.conv_2(x))
        x = self.relu(self.conv_3(x))
        x = x.view(x.size(0), -1)
        x = F.selu(self.linear_0(x))
        return self.linear_1(x), self.linear_2(x)

    def sampling(self, z_mean, z_logvar):
        epsilon = 1e-2 * torch.randn_like(z_logvar)
        return torch.exp(0.5 * z_logvar) * epsilon + z_mean

    def decode(self, z):
        z = F.selu(self.linear_3(z))
        z = z.view(z.size(0), 1, z.size(-1)).repeat(1, 120, 1)
        output, hn = self.gru(z)
        out_reshape = output.contiguous().view(-1, output.size(-1))
        y0 = F.softmax(self.linear_4(out_reshape), dim=1)
        y = y0.contiguous().view(output.size(0), -1, y0.size(-1))
        return y

    def forward(self, x):
        z_mean, z_logvar = self.encode(x)
        z = self.sampling(z_mean, z_logvar)
        return self.decode(z), z_mean, z_logvar


def load_dataset_chunked(filename, split=True, batch_size=10000):
    # Open the HDF5 file explicitly
    h5f = h5py.File(filename, 'r')

    # Memory-mapping the data (this avoids loading the entire dataset into memory at once)
    data_test = np.array(h5f['data_test'], dtype='float32', copy=False)
    
    # Handle charset as strings directly
    charset = h5f['charset']
    if charset.dtype.kind in {'S', 'O'}:  # If it's a string or object type
        charset = [x.decode('utf-8') if isinstance(x, bytes) else x for x in charset]  # Decode bytes if needed
    else:
        charset = np.array(charset, dtype='float32', copy=False)
    
    if split:
        # Instead of loading the entire data_train, we'll iterate in chunks
        data_train = h5f['data_train']
        total_samples = data_train.shape[0]
        
        # Define the generator that reads data in chunks
        def data_batch_generator():
            """Generator to load data in batches."""
            for i in range(0, total_samples, batch_size):
                batch = data_train[i:i+batch_size]  # Read a batch from disk
                yield batch

        # Return the generator, data_test, and charset
        return (data_batch_generator(), data_test, charset)
    else:
        # If not splitting, return data_test and charset only
        return (data_test, charset)
    
    # Don't forget to close the file manually when done
    h5f.close()

In [ ]:
import time
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = MolecularVAE().to(device)

torch.manual_seed(42)

class dotdict(dict): 
  __getattr__ = dict.get
  __setattr__ = dict.__setitem__
  __delattr__ = dict.__delitem__

args = dotdict()
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
checkpoint = torch.load(PATH, weights_only=True)
model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']
train_losses_p = checkpoint['train_losses_p']
val_losses_p = checkpoint['val_losses_p']

model.eval()

In [ ]:
_, _, charset = load_dataset(charset_path)

In [ ]:
training_set_org = pd.read_csv(trainingset_path)

In [ ]:
training_set_org.describe()

In [ ]:
latent_data = pd.read_csv(latent_dataset_path)

In [ ]:
x = np.float32(latent_data)
y = np.float32(training_set_org[['QED']])#, 'SA_score']])

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, representation, values):
        'Initialization'
        self.lines = representation
        self.values = values

  def __len__(self):
        'Denotes the total number of samples'
        return self.values.shape[0]

  def __getitem__(self, index):
        'Generates one sample of data'
        y = y_train[index,:]
        x = x_train[index,:]
        return x, y

In [ ]:
    data_s1 = training_set_org.sort_values(by=['QED']).dropna(subset=['QED']).drop_duplicates(subset=['QED']).tail(100)
#    data_s2 = training_set_org.sort_values(by=['SA_score'], ascending=True).dropna(subset=['SA_score']).drop_duplicates(subset=['SA_score']).head(100)
    data_s1 = data_s1['QED'].reset_index(drop=True)
#    data_s2 = data_s2['SA_score'].reset_index(drop=True)
#    y_s = np.float32(pd.concat([data_s1, data_s2], axis=1, ignore_index=True))
    y_s = np.float32(data_s1)
    y_s = torch.tensor(y_s).float()

In [ ]:
    args.batch_size = 100

In [ ]:
    from sklearn.model_selection import train_test_split

    train_ratio = 0.80
    validation_ratio = 0.20
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=1 - train_ratio, random_state=42)

    # Data container:
    training_set = Dataset(y_train, x_train)
    validation_set = Dataset(y_val, x_val)
    train_loader = torch.utils.data.DataLoader(dataset=training_set,  batch_size=args.batch_size,shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=validation_set, batch_size=args.batch_size, shuffle=False)

    print('train: ', len(training_set), ', test: ' , len(validation_set), 'sample len: ', len(y_s))

In [ ]:
    flow_train_losses = []
    flow_val_losses = []

    #num_shared_embedding = 50
    num_layers = 4
    hidden_features = 32

In [ ]:
# https://github.com/bayesiains/nflows/blob/master/examples/moons.ipynb
from nflows import transforms, distributions, flows

base_dist = distributions.StandardNormal(shape=[x_train.shape[1]])#, context=[y_train.shape[1]])
transform = []
for _ in range(num_layers):
    transform.append(transforms.autoregressive.MaskedAffineAutoregressiveTransform(features=x_train.shape[1], 
                                                                     hidden_features=hidden_features))
    transform.append(transforms.permutations.RandomPermutation(features=x_train.shape[1]))

transform_list = transforms.base.CompositeTransform(transform)

flow = flows.base.Flow(transform_list, base_dist)

In [ ]:
    args.learning_rate = 1.e-3
    args.n_epochs = 10000
    last_epoch = len(flow_train_losses)
    args.log_interval2 = 500 # (args.n_epochs-last_epoch)/100

    # Size and network parameters
    pytorch_total_params_grad = sum(p.numel() for p in flow.parameters() if p.requires_grad)
    print('Total params to optimize:', pytorch_total_params_grad)

    optimizer = optim.Adam(flow.parameters(),
                           lr=args.learning_rate)#, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-4, T_max=args.n_epochs)
    #scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[args.n_epochs/5*2,args.n_epochs/5*4], gamma=0.1)

    torch.cuda.empty_cache()
    flow = flow.cuda()
    
    time0 = time.time()

    for epoch in range(last_epoch +1, args.n_epochs + 1):
        flow.train()
        flow_train_loss = 0
        for batch_idx, (params, datas) in enumerate(train_loader):
            optimizer.zero_grad()
            params = params.to(device)
            #datas = datas.reshape(-1, 1).to(device)
            #print('shapes', params.shape, datas.shape)
            loss = -flow.log_prob(inputs=params).mean() #, context=datas).mean()
            #loss = -flow.log_prob(inputs=params, context=datas).mean()
            loss.backward()
            optimizer.step()
            scheduler.step()
            flow_train_loss += loss.item()

        flow_train_loss_avg = flow_train_loss / len(train_loader) 
        
        # Validation set
        flow.eval()
        flow_val_loss = 0
        for batch_idx, (val_params, val_data) in enumerate(test_loader):
            #val_data = val_data.to(device)
            val_params = val_params.to(device)
            val_loss = -flow.log_prob(inputs=val_params).mean()#, context=val_data).mean()
            #val_loss = -flow.log_prob(inputs=val_params, context=val_data).mean()
            flow_val_loss += val_loss.item()

        flow_val_loss_avg = flow_val_loss / len(test_loader)

        flow_val_losses.append(flow_val_loss_avg)
        flow_train_losses.append(flow_train_loss_avg)

        time_it = (time.time()-time0)/60.
        
        if epoch % args.log_interval2 ==0: print('====> Epoch: {} Average loss: {:.4f} Validation loss: {:.4f} Time: {:.3f} min'.format(epoch, flow_train_loss_avg, flow_val_loss_avg, time_it))
            
    print('Training: {0:2.2f} min'.format( (time.time()-time0)/60.))

In [ ]:
PATH = "MaskedAffineAutoregressiveTransform-4-32.pth"

torch.save({
            'epoch': epoch,
            'model_state_dict': flow.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_losses_p': flow_train_losses,
            'val_losses_p': flow_val_losses,
            }, PATH)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Convert tensors to regular Python numbers (floats)
df_losses_dict = {
    'train': [loss.item() if isinstance(loss, torch.Tensor) else loss for loss in flow_train_losses],
    'val': [loss.item() if isinstance(loss, torch.Tensor) else loss for loss in flow_val_losses]
}

# Now you can create the DataFrame
df_losses = pd.DataFrame(df_losses_dict)
# Continue with plotting
sns.set_style("whitegrid")
sns.lineplot(data=df_losses)  # Plots the training and validation losses
plt.ylabel('Losses')
#plt.ylim(-1800, -500)

In [ ]:
    cycles_of_generated_samples = 100 # 
    how_many = 10 # width
    range_range = 100 # length of the seeding vector
    total_samples = range_range * how_many * cycles_of_generated_samples                                                                                                                                                          
    time0 = time.time()
    
    print('Number of samples is ', range_range, 'x ',cycles_of_generated_samples, 'x ', how_many)

    flow.eval()


    conditional = False
    
    with torch.no_grad():
        for i in range(range_range):
            counter = 0
            while counter < cycles_of_generated_samples:
                if conditional == True:    
                    conditional_tensor = y_s.to(device).float().unsqueeze(1)
                    #sample = flow.sample(len(conditional_tensor), context=conditional_tensor)
                    sample = flow.sample(how_many, context=conditional_tensor)
                else:
                    sample = flow.sample(how_many)
                sample = np.array(sample.cpu())
                if counter == 0 and i == 0:
                    samples = sample
                else:
                    samples = np.vstack([samples, sample])
                counter += 1
            print('sample', i, ', {0:2.2f} min'.format( (time.time()-time0)/60.), samples.shape)

In [ ]:
    #samples_transformed = np.array(samples)
    samples = np.array(samples)#.cpu())
    try:
        samples_transformed = samples[:, :] # those generated from high QED
#        samples_2 = samples[:, 1, :] # those generated of the same lengths
#        samples_transformed = np.concatenate((samples_1, samples_2), axis=0) # 50 min for 200 samples
        print(samples_transformed.shape, 'Conditional sampling: {0:2.2f} min'.format( (time.time()-time0)/60.))
    except:
        print(samples_transformed.shape, 'Sampling: {0:2.2f} min'.format( (time.time()-time0)/60.))

In [ ]:
torch.cuda.empty_cache()
samples_transformed_re = samples.reshape(-1, 292)#.head(32000)
samples_transformed_re.shape

In [ ]:
number_of_splits = samples.shape[0]/500
split = np.array_split(samples, number_of_splits, axis=0)  # Split into X chunks along rows

def process_decoding(part):
    return np.float32(model.decode(torch.tensor(part).float().cuda()).cpu()) # decode in segments

model.eval()
with torch.no_grad():
    processed_part = [process_decoding(iterator) for iterator in split] # apply function

samples_decoded = np.vstack(processed_part) # combined together 

In [ ]:
all_smiles = []
valid_smiles = []

for id, molecule in enumerate(samples_decoded):
    all_smiles.append(decode_smiles_from_indexes(molecule.reshape(1, 120, len(charset)).argmax(axis=2)[0], charset))

for smi in all_smiles:
    m = Chem.MolFromSmiles(smi,sanitize=False)
    if m is None:
        pass
    else:
        try:
            Chem.SanitizeMol(m)
            valid_smiles.append(smi)
        except:
            pass

print('%.2f' % (len(valid_smiles) / len(all_smiles)*100),  '% of generated samples are valid samples, where all smiles is ', len(all_smiles))

In [ ]:
all_smiles[10]

In [ ]:
from collections import OrderedDict
unique_smiles = OrderedDict((x, True) for x in valid_smiles).keys()
print('%.2f' % (len(unique_smiles) / len(all_smiles)*100),  '% of generated samples are unique samples.')

In [ ]:
print('Calculating QED/SAS for ', len(unique_smiles), ' molecules out of all samples')
df_generated = pd.DataFrame(unique_smiles, columns=["Original_SMILES"]).drop_duplicates(subset=['Original_SMILES'])
for index, row in df_generated.iterrows():
        try:
            mol = Chem.MolFromSmiles(row['Original_SMILES'])#+'OP(C)(=O)F')
            qed = QED.default(mol)
            try:
                sas_score = sascorer.calculateScore(mol)
            except:
                sas_score = np.nan
        except:
            sas_score = np.nan
            qed = np.nan
        
        df_generated.at[index, "QED"] = qed
        df_generated.at[index, "SA_score"] = sas_score
        df_generated.at[index, "Origin"] = str(cycle)+'_iter'

new = df_generated.dropna(subset=['QED', 'SA_score']).sort_values(['QED'], ascending=False)
print('%.2f' % (len(new) / len(all_smiles)*100),  '% of generated samples are unique samples with QED/SAS scores.')

In [ ]:
new

In [ ]:
named = 'MaskedAffineAutoregressive-4-32.csv'
#new.to_csv(named, index=False)

# Are these datapoints new?

In [ ]:
import pandas as pd
all_examples = pd.read_csv(seeding_dataset, index_col=0)
all_examples['Origin'] = "ChEMBL22-50k"

In [ ]:
duplicates = pd.merge(all_examples, new, on=['Original_SMILES'], how='inner')
print('out of ', len(unique_smiles), ' unique samples', len(duplicates), 'were present in the original dataset')

In [ ]:
print('previous dataset had ', len(all_examples), ' data points')
formed_new_dataset = pd.merge(all_examples, new, on=['Original_SMILES', 'QED', 'SA_score', 'Origin'], how='outer').drop_duplicates(subset=['Original_SMILES'], keep='first')
formed_new_dataset.to_csv(str(cycle)+'_'+named, index=False)

In [ ]:
formed_new_dataset

In [ ]:
formed_new_dataset.sort_values(['QED'], ascending=False)